In [2]:
# load ztf data
from pathlib import Path
from datetime import datetime
import pandas as pd
import lasair
import numpy as np
from utils import processLasairData

project_root = Path.cwd().parent
folder_name = input("Enter name for run folder (leave blank to use timestamp): ").strip()
if not folder_name:
    start_time = datetime.now()
    folder_name = start_time.strftime("%Y-%m-%d_%H-%M-%S")
run_folder = project_root / "runs" / folder_name
run_folder.mkdir(parents=True, exist_ok=True)

ztf_data = project_root / "ztf_cleansed.csv"

ztf_df = pd.read_csv(ztf_data)

print(ztf_df.head())

Enter name for run folder (leave blank to use timestamp): test2
          ZTFID      IAUID           RA          Dec     peakt peakfilt  \
0  ZTF17aabtvsy  SN2022yei  10:35:32.09  +37:38:59.0  59870.49        r   
1  ZTF17aabvong  SN2024xxq  02:05:07.68  +11:14:55.1  60606.25        g   
2  ZTF17aacldgo  SN2022zxv  03:09:24.36  -04:53:39.2  59897.25        g   
3  ZTF17aadlxmv  SN2020adv  08:29:47.59  +33:54:22.8  58879.19        g   
4  ZTF18aaaonon  SN2022jjs  10:19:05.51  +14:24:16.6  59703.27        g   

   peakmag  peakabs duration    rise      fade   type  redshift          b  \
0  18.0303   -19.41  >34.229   >6.01    28.219  SN Ia   0.06922  59.641962   
1  16.8039   -19.57   23.222   8.464    14.758  SN Ia   0.03400 -47.664064   
2  18.7979   -18.91    >1077   >3.85  >1073.15  SN Ia   0.07200 -50.332472   
3  17.9475   -19.34   25.146  10.951    14.195  SN Ia   0.06200  34.174702   
4  18.5663   -19.08  >18.631   3.691    >14.94  SN Ia   0.07141  52.363911   

     A_V  
0  0.

In [ ]:
from typing import Any

lasair_token = 'c4ce6509f05363cfb57aaedffb056ef31573e647'
client = lasair.lasair_client(lasair_token)
print("Lasair client initialized.")

num_total = len(ztf_df)
num_processed = 0
num_success = 0

for idx, row in enumerate[Any](ztf_df.itertuples(index=False)):
    num_processed += 1
    print(f"\nProcessing {num_processed:3d}/{num_total}: {row.ZTFID}...", end='')
    ztf_id = row.ZTFID
    iauid = row.IAUID
    if str(iauid).startswith('SN'):
        iauid = str(iauid)[2:]

    try:
        result = client.object(ztf_id)
    except Exception as e:
        print(f"\nLasair API error for {ztf_id}: {e}")
        continue

    if not result or 'candidates' not in result:
        print(f"\nNo candidates found for {ztf_id}. The object may not exist or API response malformed.")
        continue
    if "error" in result:
        print(f"\nAPI returned error for {ztf_id}: {result.get('error')}")
        continue

    candidates = result['candidates']
    if not candidates:
        print(f"\nNo light curve data for {ztf_id}")
        continue

    output_rows = []
    for cand in candidates:
        # fid: filter ID (1=g, 2=r)
        fid = cand.get("fid")
        fid_to_letter = {1: "g", 2: "r", 3: "i"}
        filter_letter = fid_to_letter.get(fid, fid) if fid is not None else None

        out_row = {
            'ztf_id': ztf_id,
            'iauid': iauid,
            'MJD': cand.get('mjd', None),
            'filter': filter_letter,
            'unforced_mag': cand.get('magpsf', None),
            'unforced_mag_error': cand.get('sigmapsf', None),
            'unforced_mag_status': 'positive' if cand.get('isdiffpos', 't') == 't' else 'negative',
            'forced_ujy': cand.get('forcediffimflux', None),
            'forced_ujy_error': cand.get('forcediffimfluxunc', None),
        }
        output_rows.append(out_row)

    # Save per-object light curve CSV in run_folder (use ZTF ID for filename)
    df_obj = pd.DataFrame(output_rows)
    out_path = run_folder / f"{ztf_id}_lightcurve.csv"
    df_obj.to_csv(out_path, index=False)
    print(f" Saved {len(output_rows)} rows to {out_path.name}.", end=' ')
    try:
        processLasairData(df_obj, out_path)
        num_success += 1
        print(f"Done ({num_success}/{num_processed})")
    except Exception as e:
        print(f"Error running processLasairData: {e} ({num_success}/{num_processed})")

print(f"\nAll done! {num_success} of {num_total} objects processed successfully.")

Lasair client initialized.
Saved 60 rows to ZTF17aabtvsy_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF17aabtvsy_lightcurve.csv (23 rows)
Saved 85 rows to ZTF17aabvong_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF17aabvong_lightcurve.csv (31 rows)
Saved 74 rows to ZTF17aacldgo_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF17aacldgo_lightcurve.csv (17 rows)
Saved 71 rows to ZTF17aadlxmv_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF17aadlxmv_lightcurve.csv (28 rows)
Saved 105 rows to ZTF18aaaonon_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aaaonon_lightcurve.csv (13 rows)
Saved 70 rows to ZTF18aaaooqj_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aaa

Saved 511 rows to ZTF18aaqkqxi_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aaqkqxi_lightcurve.csv (183 rows)
Saved 113 rows to ZTF18aaqldvl_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aaqldvl_lightcurve.csv (42 rows)
Saved 57 rows to ZTF18aaqnxjb_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aaqnxjb_lightcurve.csv (21 rows)
Saved 43 rows to ZTF18aaqpeoq_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aaqpeoq_lightcurve.csv (23 rows)
Saved 822 rows to ZTF18aaqqhis_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aaqqhis_lightcurve.csv (233 rows)
Saved 25 rows to ZTF18aaqskoy_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aaqskoy_lightcurve.csv (13

Saved 33 rows to ZTF18aawxahl_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aawxahl_lightcurve.csv (12 rows)
Saved 88 rows to ZTF18aawzens_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aawzens_lightcurve.csv (41 rows)
Saved 898 rows to ZTF18aaxcfbt_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aaxcfbt_lightcurve.csv (451 rows)
Saved 99 rows to ZTF18aaxcntm_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aaxcntm_lightcurve.csv (36 rows)
Saved 34 rows to ZTF18aaxdrjn_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aaxdrjn_lightcurve.csv (34 rows)
Saved 321 rows to ZTF18aaxeoqk_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18aaxeoqk_lightcurve.csv (66 r

Saved 24 rows to ZTF18abdfwur_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abdfwur_lightcurve.csv (24 rows)
Saved 19 rows to ZTF18abdfydj_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abdfydj_lightcurve.csv (19 rows)
Saved 10 rows to ZTF18abdgwxn_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abdgwxn_lightcurve.csv (10 rows)
Saved 31 rows to ZTF18abdiwpt_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abdiwpt_lightcurve.csv (31 rows)
Saved 19 rows to ZTF18abdkimx_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abdkimx_lightcurve.csv (19 rows)
Saved 125 rows to ZTF18abdnffn_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abdnffn_lightcurve.csv (48 row

Saved 17 rows to ZTF18ablqkud_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18ablqkud_lightcurve.csv (17 rows)
Saved 17 rows to ZTF18ablqlzp_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18ablqlzp_lightcurve.csv (17 rows)
Saved 18 rows to ZTF18ablqqqp_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18ablqqqp_lightcurve.csv (18 rows)
Saved 33 rows to ZTF18ablrljh_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18ablrljh_lightcurve.csv (33 rows)
Saved 19 rows to ZTF18abltaxf_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18abltaxf_lightcurve.csv (19 rows)
Saved 12 rows to ZTF18ablwtkf_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18ablwtkf_lightcurve.csv (12 rows

Saved 39 rows to ZTF18acaezsx_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acaezsx_lightcurve.csv (11 rows)
Saved 23 rows to ZTF18acbvcmw_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acbvcmw_lightcurve.csv (17 rows)
Saved 33 rows to ZTF18acbvgqw_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acbvgqw_lightcurve.csv (21 rows)
Saved 19 rows to ZTF18acbxmvy_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acbxmvy_lightcurve.csv (15 rows)
Saved 20 rows to ZTF18acbxsap_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acbxsap_lightcurve.csv (12 rows)
Saved 22 rows to ZTF18acbxsge_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acbxsge_lightcurve.csv (19 rows

Saved 30 rows to ZTF18acspjqd_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acspjqd_lightcurve.csv (11 rows)
Saved 137 rows to ZTF18acsremz_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acsremz_lightcurve.csv (55 rows)
Saved 23 rows to ZTF18acsxpmp_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acsxpmp_lightcurve.csv (11 rows)
Saved 17 rows to ZTF18acsxusq_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acsxusq_lightcurve.csv (7 rows)
Saved 27 rows to ZTF18acsyanq_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acsyanq_lightcurve.csv (15 rows)
Saved 35 rows to ZTF18acsyzca_lightcurve.csv
Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF18acsyzca_lightcurve.csv (20 rows